# Import Dependencies

In [121]:
# !pip install -U googlemaps

In [122]:
# Dependencies and Setup FOR API
import pandas as pd
import sqlite3 as sql 
from sqlalchemy import create_engine, inspect
import numpy as np
import requests
import time
import datetime as dt
import json

import googlemaps
#from api_keys import g_key

from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session


## Read and Store Data Science Jobs Salaries CSV file into DataFrame

In [123]:
# Read and Store linkedin_jobs_usa file into dataframe 
jobs_stats = "data/ds_salary.csv"
jobs_stats_df = pd.read_csv(jobs_stats, index_col=0)
jobs_stats_df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L


## Transform the Dataframe

In [124]:
# Check the null values (if any) in the df
jobs_stats_df.isnull().sum()

work_year             0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
employee_residence    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64

In [125]:
# Check for duplicates
jobs_stats_df.duplicated().sum()

42

In [126]:
# Check the unique job titles in the df
print(jobs_stats_df['job_title'].nunique())
print(jobs_stats_df['job_title'].unique())

50
['Data Scientist' 'Machine Learning Scientist' 'Big Data Engineer'
 'Product Data Analyst' 'Machine Learning Engineer' 'Data Analyst'
 'Lead Data Scientist' 'Business Data Analyst' 'Lead Data Engineer'
 'Lead Data Analyst' 'Data Engineer' 'Data Science Consultant'
 'BI Data Analyst' 'Director of Data Science' 'Research Scientist'
 'Machine Learning Manager' 'Data Engineering Manager'
 'Machine Learning Infrastructure Engineer' 'ML Engineer' 'AI Scientist'
 'Computer Vision Engineer' 'Principal Data Scientist'
 'Data Science Manager' 'Head of Data' '3D Computer Vision Researcher'
 'Data Analytics Engineer' 'Applied Data Scientist'
 'Marketing Data Analyst' 'Cloud Data Engineer' 'Financial Data Analyst'
 'Computer Vision Software Engineer' 'Director of Data Engineering'
 'Data Science Engineer' 'Principal Data Engineer'
 'Machine Learning Developer' 'Applied Machine Learning Scientist'
 'Data Analytics Manager' 'Head of Data Science' 'Data Specialist'
 'Data Architect' 'Finance Data A

In [128]:
# Check the unique countries in the df
print(jobs_stats_df['company_location'].nunique())
print(jobs_stats_df['company_location'].unique())

50
['DE' 'JP' 'GB' 'HN' 'US' 'HU' 'NZ' 'FR' 'IN' 'PK' 'CN' 'GR' 'AE' 'NL'
 'MX' 'CA' 'AT' 'NG' 'ES' 'PT' 'DK' 'IT' 'HR' 'LU' 'PL' 'SG' 'RO' 'IQ'
 'BR' 'BE' 'UA' 'IL' 'RU' 'MT' 'CL' 'IR' 'CO' 'MD' 'KE' 'SI' 'CH' 'VN'
 'AS' 'TR' 'CZ' 'DZ' 'EE' 'MY' 'AU' 'IE']


In [129]:
# Rename the cells of remote_ratio column as 'onsite', 'hybrid' ,'remote' for plotting
jobs_stats_df.remote_ratio.replace([0,50,100], ['onsite', 'hybrid' ,'remote'],inplace = True)
jobs_stats_df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,onsite,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,onsite,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,hybrid,GB,M
3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,onsite,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,hybrid,US,L


In [131]:
# Write the df to SQLite database
conn = sql.connect('data/jobstats_db.sqlite')
jobs_stats_df.to_sql('job_stats', conn, if_exists='replace', index=False)
pd.read_sql('select * from job_stats', conn)

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,onsite,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,onsite,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,hybrid,GB,M
3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,onsite,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,hybrid,US,L
...,...,...,...,...,...,...,...,...,...,...,...
602,2022,SE,FT,Data Engineer,154000,USD,154000,US,remote,US,M
603,2022,SE,FT,Data Engineer,126000,USD,126000,US,remote,US,M
604,2022,SE,FT,Data Analyst,129000,USD,129000,US,onsite,US,M
605,2022,SE,FT,Data Analyst,150000,USD,150000,US,remote,US,M


In [132]:
pd.read_sql('select count(*), job_title from job_stats group by job_title', conn)

,count(*),job_title
0,1,3D Computer Vision Researcher
1,7,AI Scientist
2,4,Analytics Engineer
3,5,Applied Data Scientist
4,4,Applied Machine Learning Scientist
5,6,BI Data Analyst
6,1,Big Data Architect
7,8,Big Data Engineer
8,5,Business Data Analyst
9,2,Cloud Data Engineer


In [133]:
conn = sql.connect('data/jobstats_db.sqlite')
cur = conn.cursor()

In [142]:
word_data = cur.execute('select count(*),job_title from job_stats group by job_title order by count(*) desc').fetchall()

In [135]:
word_data

[(143, 'Data Scientist'),
 (132, 'Data Engineer'),
 (97, 'Data Analyst'),
 (41, 'Machine Learning Engineer'),
 (16, 'Research Scientist'),
 (12, 'Data Science Manager'),
 (11, 'Data Architect'),
 (8, 'Machine Learning Scientist'),
 (8, 'Big Data Engineer'),
 (7, 'Principal Data Scientist'),
 (7, 'Director of Data Science'),
 (7, 'Data Science Consultant'),
 (7, 'Data Analytics Manager'),
 (7, 'AI Scientist'),
 (6, 'ML Engineer'),
 (6, 'Lead Data Engineer'),
 (6, 'Computer Vision Engineer'),
 (6, 'BI Data Analyst'),
 (5, 'Head of Data'),
 (5, 'Data Engineering Manager'),
 (5, 'Business Data Analyst'),
 (5, 'Applied Data Scientist'),
 (4, 'Head of Data Science'),
 (4, 'Data Analytics Engineer'),
 (4, 'Applied Machine Learning Scientist'),
 (4, 'Analytics Engineer'),
 (3, 'Principal Data Engineer'),
 (3, 'Machine Learning Infrastructure Engineer'),
 (3, 'Machine Learning Developer'),
 (3, 'Lead Data Scientist'),
 (3, 'Lead Data Analyst'),
 (3, 'Data Science Engineer'),
 (3, 'Computer Visi

In [136]:
jsonString = json.dumps(word_data)

In [137]:
jsonString

'[[143, "Data Scientist"], [132, "Data Engineer"], [97, "Data Analyst"], [41, "Machine Learning Engineer"], [16, "Research Scientist"], [12, "Data Science Manager"], [11, "Data Architect"], [8, "Machine Learning Scientist"], [8, "Big Data Engineer"], [7, "Principal Data Scientist"], [7, "Director of Data Science"], [7, "Data Science Consultant"], [7, "Data Analytics Manager"], [7, "AI Scientist"], [6, "ML Engineer"], [6, "Lead Data Engineer"], [6, "Computer Vision Engineer"], [6, "BI Data Analyst"], [5, "Head of Data"], [5, "Data Engineering Manager"], [5, "Business Data Analyst"], [5, "Applied Data Scientist"], [4, "Head of Data Science"], [4, "Data Analytics Engineer"], [4, "Applied Machine Learning Scientist"], [4, "Analytics Engineer"], [3, "Principal Data Engineer"], [3, "Machine Learning Infrastructure Engineer"], [3, "Machine Learning Developer"], [3, "Lead Data Scientist"], [3, "Lead Data Analyst"], [3, "Data Science Engineer"], [3, "Computer Vision Software Engineer"], [2, "Pr

In [143]:
num_jobs = cur.execute('select company_location, COUNT(*) from job_stats GROUP BY company_location').fetchall()

In [139]:
num_jobs

[('AE', 3),
 ('AS', 1),
 ('AT', 4),
 ('AU', 3),
 ('BE', 2),
 ('BR', 3),
 ('CA', 30),
 ('CH', 2),
 ('CL', 1),
 ('CN', 2),
 ('CO', 1),
 ('CZ', 2),
 ('DE', 28),
 ('DK', 3),
 ('DZ', 1),
 ('EE', 1),
 ('ES', 14),
 ('FR', 15),
 ('GB', 47),
 ('GR', 11),
 ('HN', 1),
 ('HR', 1),
 ('HU', 1),
 ('IE', 1),
 ('IL', 1),
 ('IN', 24),
 ('IQ', 1),
 ('IR', 1),
 ('IT', 2),
 ('JP', 6),
 ('KE', 1),
 ('LU', 3),
 ('MD', 1),
 ('MT', 1),
 ('MX', 3),
 ('MY', 1),
 ('NG', 2),
 ('NL', 4),
 ('NZ', 1),
 ('PK', 3),
 ('PL', 4),
 ('PT', 4),
 ('RO', 1),
 ('RU', 2),
 ('SG', 1),
 ('SI', 2),
 ('TR', 3),
 ('UA', 1),
 ('US', 355),
 ('VN', 1)]

In [140]:
num_jobs_list = []

for country, job_count in num_jobs:
    num_jobs_dict = {}
    num_jobs_dict["Name"] = country
    num_jobs_dict["Count"] = job_count
    num_jobs_list.append(num_jobs_dict)


In [116]:
bubble_dict ={}
bubble_dict["children"] = num_jobs_list
bubble_dict

{'children': [{'Name': 'AE', 'Count': 3},
  {'Name': 'AS', 'Count': 1},
  {'Name': 'AT', 'Count': 4},
  {'Name': 'AU', 'Count': 3},
  {'Name': 'BE', 'Count': 2},
  {'Name': 'BR', 'Count': 3},
  {'Name': 'CA', 'Count': 30},
  {'Name': 'CH', 'Count': 2},
  {'Name': 'CL', 'Count': 1},
  {'Name': 'CN', 'Count': 2},
  {'Name': 'CO', 'Count': 1},
  {'Name': 'CZ', 'Count': 2},
  {'Name': 'DE', 'Count': 28},
  {'Name': 'DK', 'Count': 3},
  {'Name': 'DZ', 'Count': 1},
  {'Name': 'EE', 'Count': 1},
  {'Name': 'ES', 'Count': 14},
  {'Name': 'FR', 'Count': 15},
  {'Name': 'GB', 'Count': 47},
  {'Name': 'GR', 'Count': 11},
  {'Name': 'HN', 'Count': 1},
  {'Name': 'HR', 'Count': 1},
  {'Name': 'HU', 'Count': 1},
  {'Name': 'IE', 'Count': 1},
  {'Name': 'IL', 'Count': 1},
  {'Name': 'IN', 'Count': 24},
  {'Name': 'IQ', 'Count': 1},
  {'Name': 'IR', 'Count': 1},
  {'Name': 'IT', 'Count': 2},
  {'Name': 'JP', 'Count': 6},
  {'Name': 'KE', 'Count': 1},
  {'Name': 'LU', 'Count': 3},
  {'Name': 'MD', 'Cou

In [141]:
country_jobs_dict = bubble_dict
country_jobs_dict


{'children': [{'Name': 'AE', 'Count': 3},
  {'Name': 'AS', 'Count': 1},
  {'Name': 'AT', 'Count': 4},
  {'Name': 'AU', 'Count': 3},
  {'Name': 'BE', 'Count': 2},
  {'Name': 'BR', 'Count': 3},
  {'Name': 'CA', 'Count': 30},
  {'Name': 'CH', 'Count': 2},
  {'Name': 'CL', 'Count': 1},
  {'Name': 'CN', 'Count': 2},
  {'Name': 'CO', 'Count': 1},
  {'Name': 'CZ', 'Count': 2},
  {'Name': 'DE', 'Count': 28},
  {'Name': 'DK', 'Count': 3},
  {'Name': 'DZ', 'Count': 1},
  {'Name': 'EE', 'Count': 1},
  {'Name': 'ES', 'Count': 14},
  {'Name': 'FR', 'Count': 15},
  {'Name': 'GB', 'Count': 47},
  {'Name': 'GR', 'Count': 11},
  {'Name': 'HN', 'Count': 1},
  {'Name': 'HR', 'Count': 1},
  {'Name': 'HU', 'Count': 1},
  {'Name': 'IE', 'Count': 1},
  {'Name': 'IL', 'Count': 1},
  {'Name': 'IN', 'Count': 24},
  {'Name': 'IQ', 'Count': 1},
  {'Name': 'IR', 'Count': 1},
  {'Name': 'IT', 'Count': 2},
  {'Name': 'JP', 'Count': 6},
  {'Name': 'KE', 'Count': 1},
  {'Name': 'LU', 'Count': 3},
  {'Name': 'MD', 'Cou

In [119]:
country_jobs_dict = {'children': [{'Name': 'AE', 'Count': 3},
  {'Name': 'AS', 'Count': 1},
  {'Name': 'AT', 'Count': 4},
  {'Name': 'AU', 'Count': 3},
  {'Name': 'BE', 'Count': 2},
  {'Name': 'BR', 'Count': 3},
  {'Name': 'CA', 'Count': 30},
  {'Name': 'CH', 'Count': 2},
  {'Name': 'CL', 'Count': 1},
  {'Name': 'CN', 'Count': 2},
  {'Name': 'CO', 'Count': 1},
  {'Name': 'CZ', 'Count': 2},
  {'Name': 'DE', 'Count': 28},
  {'Name': 'DK', 'Count': 3},
  {'Name': 'DZ', 'Count': 1},
  {'Name': 'EE', 'Count': 1},
  {'Name': 'ES', 'Count': 14},
  {'Name': 'FR', 'Count': 15},
  {'Name': 'GB', 'Count': 47},
  {'Name': 'GR', 'Count': 11},
  {'Name': 'HN', 'Count': 1},
  {'Name': 'HR', 'Count': 1},
  {'Name': 'HU', 'Count': 1},
  {'Name': 'IE', 'Count': 1},
  {'Name': 'IL', 'Count': 1},
  {'Name': 'IN', 'Count': 24},
  {'Name': 'IQ', 'Count': 1},
  {'Name': 'IR', 'Count': 1},
  {'Name': 'IT', 'Count': 2},
  {'Name': 'JP', 'Count': 6},
  {'Name': 'KE', 'Count': 1},
  {'Name': 'LU', 'Count': 3},
  {'Name': 'MD', 'Count': 1},
  {'Name': 'MT', 'Count': 1},
  {'Name': 'MX', 'Count': 3},
  {'Name': 'MY', 'Count': 1},
  {'Name': 'NG', 'Count': 2},
  {'Name': 'NL', 'Count': 4},
  {'Name': 'NZ', 'Count': 1},
  {'Name': 'PK', 'Count': 3},
  {'Name': 'PL', 'Count': 4},
  {'Name': 'PT', 'Count': 4},
  {'Name': 'RO', 'Count': 1},
  {'Name': 'RU', 'Count': 2},
  {'Name': 'SG', 'Count': 1},
  {'Name': 'SI', 'Count': 2},
  {'Name': 'TR', 'Count': 3},
  {'Name': 'UA', 'Count': 1},
  {'Name': 'US', 'Count': 355},
  {'Name': 'VN', 'Count': 1}]}

In [120]:
country_jobs_dict

{'children': [{'Name': 'AE', 'Count': 3},
  {'Name': 'AS', 'Count': 1},
  {'Name': 'AT', 'Count': 4},
  {'Name': 'AU', 'Count': 3},
  {'Name': 'BE', 'Count': 2},
  {'Name': 'BR', 'Count': 3},
  {'Name': 'CA', 'Count': 30},
  {'Name': 'CH', 'Count': 2},
  {'Name': 'CL', 'Count': 1},
  {'Name': 'CN', 'Count': 2},
  {'Name': 'CO', 'Count': 1},
  {'Name': 'CZ', 'Count': 2},
  {'Name': 'DE', 'Count': 28},
  {'Name': 'DK', 'Count': 3},
  {'Name': 'DZ', 'Count': 1},
  {'Name': 'EE', 'Count': 1},
  {'Name': 'ES', 'Count': 14},
  {'Name': 'FR', 'Count': 15},
  {'Name': 'GB', 'Count': 47},
  {'Name': 'GR', 'Count': 11},
  {'Name': 'HN', 'Count': 1},
  {'Name': 'HR', 'Count': 1},
  {'Name': 'HU', 'Count': 1},
  {'Name': 'IE', 'Count': 1},
  {'Name': 'IL', 'Count': 1},
  {'Name': 'IN', 'Count': 24},
  {'Name': 'IQ', 'Count': 1},
  {'Name': 'IR', 'Count': 1},
  {'Name': 'IT', 'Count': 2},
  {'Name': 'JP', 'Count': 6},
  {'Name': 'KE', 'Count': 1},
  {'Name': 'LU', 'Count': 3},
  {'Name': 'MD', 'Cou

In [114]:
dict =  json.dumps(num_jobs_list)
dict

'[{"Name": "AE", "Count": 3}, {"Name": "AS", "Count": 1}, {"Name": "AT", "Count": 4}, {"Name": "AU", "Count": 3}, {"Name": "BE", "Count": 2}, {"Name": "BR", "Count": 3}, {"Name": "CA", "Count": 30}, {"Name": "CH", "Count": 2}, {"Name": "CL", "Count": 1}, {"Name": "CN", "Count": 2}, {"Name": "CO", "Count": 1}, {"Name": "CZ", "Count": 2}, {"Name": "DE", "Count": 28}, {"Name": "DK", "Count": 3}, {"Name": "DZ", "Count": 1}, {"Name": "EE", "Count": 1}, {"Name": "ES", "Count": 14}, {"Name": "FR", "Count": 15}, {"Name": "GB", "Count": 47}, {"Name": "GR", "Count": 11}, {"Name": "HN", "Count": 1}, {"Name": "HR", "Count": 1}, {"Name": "HU", "Count": 1}, {"Name": "IE", "Count": 1}, {"Name": "IL", "Count": 1}, {"Name": "IN", "Count": 24}, {"Name": "IQ", "Count": 1}, {"Name": "IR", "Count": 1}, {"Name": "IT", "Count": 2}, {"Name": "JP", "Count": 6}, {"Name": "KE", "Count": 1}, {"Name": "LU", "Count": 3}, {"Name": "MD", "Count": 1}, {"Name": "MT", "Count": 1}, {"Name": "MX", "Count": 3}, {"Name": "M

In [84]:
for country, job_count in country_data:
  num_jobs_dict["name","value"] = country,job_count

In [85]:
num_jobs_dict

{'name': ('VN',), 'value': 1, ('name', 'value'): ('VN', 1)}

In [74]:
jsonString = json.dumps(country_data)

In [75]:
jsonString

'[["AE", 3], ["AS", 1], ["AT", 4], ["AU", 3], ["BE", 2], ["BR", 3], ["CA", 30], ["CH", 2], ["CL", 1], ["CN", 2], ["CO", 1], ["CZ", 2], ["DE", 28], ["DK", 3], ["DZ", 1], ["EE", 1], ["ES", 14], ["FR", 15], ["GB", 47], ["GR", 11], ["HN", 1], ["HR", 1], ["HU", 1], ["IE", 1], ["IL", 1], ["IN", 24], ["IQ", 1], ["IR", 1], ["IT", 2], ["JP", 6], ["KE", 1], ["LU", 3], ["MD", 1], ["MT", 1], ["MX", 3], ["MY", 1], ["NG", 2], ["NL", 4], ["NZ", 1], ["PK", 3], ["PL", 4], ["PT", 4], ["RO", 1], ["RU", 2], ["SG", 1], ["SI", 2], ["TR", 3], ["UA", 1], ["US", 355], ["VN", 1]]'

In [148]:
num_jobs = cur.execute('select job_title, count(job_title), AVG(salary_in_usd) from job_stats GROUP BY job_title order by count(job_title) desc limit 10').fetchall()
num_jobs

[('Data Scientist', 143, 108187.83216783217),
 ('Data Engineer', 132, 112725.0),
 ('Data Analyst', 97, 92893.06185567011),
 ('Machine Learning Engineer', 41, 104880.14634146342),
 ('Research Scientist', 16, 109019.5),
 ('Data Science Manager', 12, 158328.5),
 ('Data Architect', 11, 177873.9090909091),
 ('Machine Learning Scientist', 8, 158412.5),
 ('Big Data Engineer', 8, 51974.0),
 ('Principal Data Scientist', 7, 215242.42857142858)]

## Read and Store USA jobs CSV file into DataFrame

In [8]:
# Read and Store linkedin_jobs_usa file into dataframe 
jobs_usa = "data/linkedin_jobs_usa.csv"
jobs_usa_df = pd.read_csv(jobs_usa)
jobs_usa_df.head()

,title,company,description,onsite_remote,salary,location,criteria,posted_date,link
0,Data Analyst - Recent Graduate,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,Buffalo-Niagara Falls Area,"[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-22,https://www.linkedin.com/jobs/view/data-analys...
1,Data Analyst - Recent Graduate,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"San Jose, CA","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-22,https://www.linkedin.com/jobs/view/data-analys...
2,Data Analyst,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"Texas, United States","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...
3,Data Analyst,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"Illinois, United States","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...
4,Entry-Level Data Analyst,The Federal Savings Bank,"The Federal Savings Bank, a national bank and ...",onsite,NaN,"Chicago, IL","[{'Seniority level': 'Entry level'}, {'Employm...",2022-11-17,https://www.linkedin.com/jobs/view/entry-level...


## Transform the DataFrame

### Add coordinates of the locations using Googlemaps

In [12]:
# gmaps = googlemaps.Client(key= g_key)

# jobs_usa_df['latitude'] = None
# jobs_usa_df['longitude'] = None

# for i in range(0, len(jobs_usa_df)):     
#     try:        
        
#         geocode_result = gmaps.geocode(jobs_usa_df.iat[i, 5])
#         lat= geocode_result[0]['geometry']['location']['lat']
#         lng= geocode_result[0]['geometry']['location']['lng']
#         jobs_usa_df.iat[i, jobs_usa_df.columns.get_loc('latitude')] = lat
#         jobs_usa_df.iat[i, jobs_usa_df.columns.get_loc('longitude')] = lng
        
#     except Exception as e:
#         print(e)
        
# jobs_usa_df  
       

list index out of range
list index out of range


,title,company,description,onsite_remote,salary,location,criteria,posted_date,link,latitude,longitude
0,Data Analyst - Recent Graduate,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,Buffalo-Niagara Falls Area,"[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-22,https://www.linkedin.com/jobs/view/data-analys...,42.851801,-78.747621
1,Data Analyst - Recent Graduate,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"San Jose, CA","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-22,https://www.linkedin.com/jobs/view/data-analys...,37.33874,-121.885253
2,Data Analyst,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"Texas, United States","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...,31.968599,-99.901813
3,Data Analyst,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"Illinois, United States","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...,40.633125,-89.398528
4,Entry-Level Data Analyst,The Federal Savings Bank,"The Federal Savings Bank, a national bank and ...",onsite,NaN,"Chicago, IL","[{'Seniority level': 'Entry level'}, {'Employm...",2022-11-17,https://www.linkedin.com/jobs/view/entry-level...,41.878114,-87.629798
...,...,...,...,...,...,...,...,...,...,...,...
2840,Junior Data Analyst,Iris Software Inc.,"Iris's client, one of the world's largest fina...",hybrid,NaN,"Texas, United States","[{'Seniority level': 'Mid-Senior level'}, {'Em...",2022-10-21,https://www.linkedin.com/jobs/view/junior-data...,31.968599,-99.901813
2841,Data Analyst (SQL),Marwood Group,The Marwood Group (Marwood) is a leading healt...,hybrid,"$75,000.00\r\n -\r\n $95...",New York City Metropolitan Area,"[{'Seniority level': 'Mid-Senior level'}, {'Em...",2022-10-26,https://www.linkedin.com/jobs/view/data-analys...,40.712776,-74.005954
2842,Data Analyst,SmartSense by Digi,"Join a high-performing, tight-knit team at a f...",hybrid,NaN,"Mishawaka, IN","[{'Seniority level': 'Associate'}, {'Employmen...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...,41.661993,-86.158616
2843,Data Analyst,Synergy Search,Nashville (Berry Hill) based company looking t...,hybrid,"$85,000.00\r\n -\r\n $95...",Nashville Metropolitan Area,"[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-14,https://www.linkedin.com/jobs/view/data-analys...,36.162646,-86.781497


### Save the updated dataframe as csv file 

In [13]:
#  Save the new dataframe as CSV file and use this file for further use so that we are not makining the api call again
# jobs_usa_df.to_csv('data/updated_jobs_usa.csv')

### Read and Store the updated CSV file into DataFrame

In [8]:
# Read and Store updated_jobs_usa file into dataframe 
updated_jobs_usa = "data/updated_jobs_usa.csv"
updated_jobs_usa_df = pd.read_csv(updated_jobs_usa, index_col=0)
updated_jobs_usa_df.head()

,title,company,description,onsite_remote,salary,location,criteria,posted_date,link,latitude,longitude
0,Data Analyst - Recent Graduate,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,Buffalo-Niagara Falls Area,"[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-22,https://www.linkedin.com/jobs/view/data-analys...,42.851801,-78.747621
1,Data Analyst - Recent Graduate,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"San Jose, CA","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-22,https://www.linkedin.com/jobs/view/data-analys...,37.338740,-121.885253
2,Data Analyst,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"Texas, United States","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...,31.968599,-99.901813
3,Data Analyst,PayPal,"At PayPal (NASDAQ: PYPL), we believe that ever...",onsite,NaN,"Illinois, United States","[{'Seniority level': 'Not Applicable'}, {'Empl...",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...,40.633125,-89.398528
4,Entry-Level Data Analyst,The Federal Savings Bank,"The Federal Savings Bank, a national bank and ...",onsite,NaN,"Chicago, IL","[{'Seniority level': 'Entry level'}, {'Employm...",2022-11-17,https://www.linkedin.com/jobs/view/entry-level...,41.878114,-87.629798


In [9]:
# Drop unwanted columns from the df
updated_jobs_usa_df = updated_jobs_usa_df.drop(columns=['criteria', 'description', 'salary'])


# Remane columns
updated_jobs_usa_df.rename(columns={'onsite_remote':'work_environment'}, inplace = True)

# Drop NaN values
updated_jobs_usa_df = updated_jobs_usa_df.dropna(subset=['latitude', 'longitude'])  

In [10]:
# Convert lattitude and longitude to float types
updated_jobs_usa_df['latitude'] = updated_jobs_usa_df['latitude'].astype(float)
updated_jobs_usa_df['longitude'] = updated_jobs_usa_df['longitude'].astype(float)

# reset index 
updated_jobs_usa_df.reset_index(drop=True)
updated_jobs_usa_df.head()




,title,company,work_environment,location,posted_date,link,latitude,longitude
0,Data Analyst - Recent Graduate,PayPal,onsite,Buffalo-Niagara Falls Area,2022-11-22,https://www.linkedin.com/jobs/view/data-analys...,42.851801,-78.747621
1,Data Analyst - Recent Graduate,PayPal,onsite,"San Jose, CA",2022-11-22,https://www.linkedin.com/jobs/view/data-analys...,37.338740,-121.885253
2,Data Analyst,PayPal,onsite,"Texas, United States",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...,31.968599,-99.901813
3,Data Analyst,PayPal,onsite,"Illinois, United States",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...,40.633125,-89.398528
4,Entry-Level Data Analyst,The Federal Savings Bank,onsite,"Chicago, IL",2022-11-17,https://www.linkedin.com/jobs/view/entry-level...,41.878114,-87.629798


In [11]:
# Write the df to SQLite database
conn = sql.connect('data/updated_jobs_usa_db.sqlite')
updated_jobs_usa_df.to_sql('updated_jobs_usa', conn, if_exists='replace', index=False)
pd.read_sql('select * from updated_jobs_usa', conn)

,title,company,work_environment,location,posted_date,link,latitude,longitude
0,Data Analyst - Recent Graduate,PayPal,onsite,Buffalo-Niagara Falls Area,2022-11-22,https://www.linkedin.com/jobs/view/data-analys...,42.851801,-78.747621
1,Data Analyst - Recent Graduate,PayPal,onsite,"San Jose, CA",2022-11-22,https://www.linkedin.com/jobs/view/data-analys...,37.338740,-121.885253
2,Data Analyst,PayPal,onsite,"Texas, United States",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...,31.968599,-99.901813
3,Data Analyst,PayPal,onsite,"Illinois, United States",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...,40.633125,-89.398528
4,Entry-Level Data Analyst,The Federal Savings Bank,onsite,"Chicago, IL",2022-11-17,https://www.linkedin.com/jobs/view/entry-level...,41.878114,-87.629798
...,...,...,...,...,...,...,...,...
2838,Junior Data Analyst,Iris Software Inc.,hybrid,"Texas, United States",2022-10-21,https://www.linkedin.com/jobs/view/junior-data...,31.968599,-99.901813
2839,Data Analyst (SQL),Marwood Group,hybrid,New York City Metropolitan Area,2022-10-26,https://www.linkedin.com/jobs/view/data-analys...,40.712776,-74.005954
2840,Data Analyst,SmartSense by Digi,hybrid,"Mishawaka, IN",2022-11-17,https://www.linkedin.com/jobs/view/data-analys...,41.661993,-86.158616
2841,Data Analyst,Synergy Search,hybrid,Nashville Metropolitan Area,2022-11-14,https://www.linkedin.com/jobs/view/data-analys...,36.162646,-86.781497


## Convert the Dataframe into Geojson and save as .js file

In [12]:
# Create a function to convert the dataframe into geojson
def df_to_geojson(df, properties, lat='latitude', lon='longitude'):
    
    # Create a new python dict to contain the geojson data, using geojson format
    geojson = {'type':'FeatureCollection', 'features':[]}

    # Loop through each row in the dataframe and convert each row to geojson format
    for _, row in df.iterrows():
        
        # Create a feature template to fill in
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}

        # Fill in the coordinates
        feature['geometry']['coordinates'] = [row[lon],row[lat]]

        # For each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]
        
        # Add this feature to the list of features inside the dict
        geojson['features'].append(feature)
    
    return geojson

In [13]:
# Pass in the dataFrame, list of columns to to the above function
cols = ['title', 'company', 'work_environment', 'location', 'posted_date', 'link']
geojson = df_to_geojson(updated_jobs_usa_df, cols)
geojson

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'title': 'Data Analyst - Recent Graduate',
    'company': 'PayPal',
    'work_environment': 'onsite',
    'location': 'Buffalo-Niagara Falls Area',
    'posted_date': '2022-11-22',
    'link': 'https://www.linkedin.com/jobs/view/data-analyst-recent-graduate-at-paypal-3364471019?refId=7aKV74baVvH%2FRwNOmaw%2ByA%3D%3D&trackingId=Wl5idVe67zjIVZhL%2BbpNOA%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card'},
   'geometry': {'type': 'Point',
    'coordinates': [-78.74762079999999, 42.8518007]}},
  {'type': 'Feature',
   'properties': {'title': 'Data Analyst - Recent Graduate',
    'company': 'PayPal',
    'work_environment': 'onsite',
    'location': 'San Jose, CA',
    'posted_date': '2022-11-22',
    'link': 'https://www.linkedin.com/jobs/view/data-analyst-recent-graduate-at-paypal-3364467390?refId=7aKV74baVvH%2FRwNOmaw%2ByA%3D%3D&trackingId=3IOGqlhBDgIdKd%2F48MX8BA%3D%3D&position=2&pageNum=

In [14]:
# Save it to a .js file for dashboard creation
output_filename = 'data/jobs_usa.js'
with open(output_filename, 'wt') as output_file:
    output_file.write('var dataset = ')
    json.dump(geojson, output_file, indent=2) 
    
    

In [ ]:
# Write geojson to a file in Python using json.dump() 

# Serializing json
json_object = json.dumps(geojson, indent=2)
 
# Writing to sample.json
with open("data/sample.json", "w") as outfile:
    outfile.write(json_object)

In [3]:
# Write the df to SQLite database
conn = sql.connect('data/sample.json.sqlite')
sample.json.to_sql('upjob_stats', conn, if_exists='replace', index=False)
# pd.read_sql('select * from job_stats', conn)

NameError: name 'sample' is not defined